# Virtual Scribe AI Comparator

The goal of this notebook is to compare the ability of various AI models to generate a Geriatrics Consultation Note

In [1]:
# Load necessary components
import os
import json

from langchain_core.prompts import PromptTemplate
from langchain_ollama import OllamaLLM

In [2]:
# Function to read notes andput into array
def read_notes_to_array(ehr):
    notes = []
    for filename in os.listdir(ehr):
        note_path = os.path.join(ehr,filename)

        if os.path.isfile(note_path) and filename.endswith(".txt"):
            with open(note_path, 'r') as note:
                notes.append(note.read())

    return notes

# Function to pull template from file
def template_generation(template):
    with open("templates/"+template+".txt", "r") as f:
        template = f.read()
    return template

In [3]:
# Pick models to test
models = [
    "llama3",
    "qwen2",
    "phi3",
    "gemma2",
    "medllama3",
    "meditron",
    "medllama2",
    "MedX3.2",
    "NewMes15"
]

# Get notes
notes = read_notes_to_array("ehr")

In [4]:
# Create the prompt
prompt = PromptTemplate(
    input_variables=["notes"],
    template=template_generation("geriatric_consult")+"Use the following information to fill out the above template: {notes}"
)

In [5]:
# Generate notes for all requested models and patients
for model in models:
    llm = OllamaLLM(model=model)
    i = 0
    for note in notes:
        i += 1
        with open("virtualscribe_output/"+model+"patient-"+str(i)+".txt", "w") as f:
            chain = prompt | llm
            f.write(json.dumps(chain.invoke(note)).replace("\\n", "\n"))
    